In [1]:
import sys
sys.path.append('../scripts')

In [2]:
import os
# Disable weights and biases (if installed)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["WANDB_DISABLED"] = "true"

In [25]:
from pathlib import Path

from dataset import load_data, get_dataloader
from evaluation import get_scores
from transformers_util import get_training_args, get_trainer, get_tokenizer

# Load Data & Seq2Seq Model

In [4]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [5]:
MODEL_PATH = "../data/ellipses/model/"
BATCH_SIZE = 32

In [6]:
tokenizer = get_tokenizer(config)

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
base_path = Path('..')
train_df, val_df, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)

# Error Analysis

In [8]:
from transformers import Text2TextGenerationPipeline, AutoModelForSeq2SeqLM
from notebook_util import calculate_errors

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

In [10]:
pipeline = Text2TextGenerationPipeline(model, tokenizer=tokenizer, max_length=config.generation_max_length, device=0, batch_size=BATCH_SIZE)

In [11]:
from notebook_util import show_errors

In [12]:
%%time
out_valid = pipeline(list(val_df.raw_sentence))

CPU times: user 54.8 s, sys: 469 ms, total: 55.2 s
Wall time: 55.2 s


In [13]:
errors_valid = calculate_errors(out_valid, val_df, tokenizer)
errors_valid.error_type.value_counts() / len(errors_valid)

tn         0.495420
tp         0.416320
replace    0.016653
delete     0.016653
fn         0.016653
insert     0.014988
fp         0.013322
complex    0.009992
Name: error_type, dtype: float64

In [27]:
%%time
scores = get_scores(errors_valid, "eval")
scores

CPU times: user 1min 20s, sys: 72.1 ms, total: 1min 20s
Wall time: 1min 21s


{'eval/exact_match': 0.9117402164862615,
 'eval/gleu': 0.9868297119100977,
 'eval/edit_distance_rel': 0.9539251318409139}

In [15]:
%%time
out_test = pipeline(list(test_df.raw_sentence))

CPU times: user 53.3 s, sys: 349 ms, total: 53.7 s
Wall time: 53.7 s


In [16]:
errors_test = calculate_errors(out_test, test_df, tokenizer)
errors_test.error_type.value_counts()

tn         601
tp         448
delete      25
insert      25
fn          18
replace     16
fp          15
complex     11
Name: error_type, dtype: int64

In [17]:
errors_test

,pred,ground_truth,original,error_type,index,file,sentence_id
0,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,tp,0,00_mundhoehlenkarzinom_0002.tsv,1
1,Bei chronischem Tabakabusus oder Alkoholabusus...,Bei chronischem Tabakabusus oder Alkoholabusus...,Bei chronischem Tabak- oder Alkoholabusus ist ...,tp,1,00_mundhoehlenkarzinom_0002.tsv,2
2,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,tp,40,00_mundhoehlenkarzinom_0098.tsv,2
3,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",tp,44,00_mundhoehlenkarzinom_0103.tsv,1
4,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",tp,47,00_mundhoehlenkarzinom_0115.tsv,4
...,...,...,...,...,...,...,...
1154,Dabei war der Anteil älterer Patient * innen u...,Dabei war der Anteil älterer Patient * innen u...,Dabei war der Anteil älterer Patient * innen u...,tn,3835,24_analkarzinom_0178.tsv,44
1155,"Ein neuer RCT (Update-Recherche 2017), dessen ...","Ein neuer RCT (Update-Recherche 2017), dessen ...","Ein neuer RCT (Update-Recherche 2017), dessen ...",tn,3840,22_prostatakarzinom_0199.tsv,46
1156,"- Clindamycin führt, völlig unabhängig von ein...","- Clindamycin führt, völlig unabhängig von ein...","- Clindamycin führt, völlig unabhängig von ein...",tn,3843,23_peniskarzinom_0158.tsv,10
1157,"Für HIV-positive Patienten hat sich gezeigt, d...","Für HIV-positive Patienten hat sich gezeigt, d...","Für HIV-positive Patienten hat sich gezeigt, d...",tn,3844,24_analkarzinom_0163.tsv,10


In [26]:
%%time
scores = get_scores(errors_test, "test")
scores

CPU times: user 1min 25s, sys: 32.1 ms, total: 1min 25s
Wall time: 1min 26s


{'test/exact_match': 0.905090595340811,
 'test/gleu': 0.9778028196418292,
 'test/edit_distance_rel': 0.950224476907634}

In [19]:
from notebook_util import show_errors

In [20]:
show_errors(errors_test[errors_test.error_type == 'insert'])

0;02_kolorektales-karzinom_0330.tsv;17

__Input:__

In die NSABP-Studie C-07 wurden 2407 Patienten im Stadium II (28,6%) oder im Stadium III eingeschlossen und erhielten entweder das Roswell-Park-Schema mit wöchentlicher 5-FU/FS Bolusgabe (3 Zyklen à jeweils 8 Wochen) oder das gleiche 5-FU/FS-Schema mit Oxaliplatin 85 mg/m2 in Woche 1, 3 und 5 im achtwöchentlichen Rhythmus (FLOX-Schema).

__Error type:__ insert

1;03_lungenkarzinom_0420.tsv;46

...

__Error type:__ insert

24;28_komplementaermedizin_0287.tsv;1

__Input:__

Tumoransprechen, - progression und -rezidive

__Error type:__ insert

# Some examples

In [21]:
def run_pipeline(examples):
    for e, p in zip(examples, pipeline(examples)):
        print(e, '-->', p['generated_text'])

In [22]:
# Forward ellipses
run_pipeline(['Krebs-Vorsorge / -Früherkennung', 'HPV31, 33, 45 und 51', 'Vitamin C, E und A1'])

Krebs-Vorsorge / -Früherkennung --> Krebs-Vorsorge / Krebs-Früherkennung
HPV31, 33, 45 und 51 --> HPV31, HPV33 und HPV51
Vitamin C, E und A1 --> Vitamin C, Vitamin E und Vitamin A1


In [23]:
# Backward ellipses
run_pipeline(['Chemo- und Strahlentherapie', 'BRAF- und MEK-Inhibitor', 'zielgerichtete und Immuntherapien'])

Chemo- und Strahlentherapie --> Chemotherapie und Strahlentherapie
BRAF- und MEK-Inhibitor --> BRAF-Inhibitor und MEK-Inhibitor
zielgerichtete und Immuntherapien --> zielgerichtete Therapien und Immuntherapien


In [24]:
# Complex ellipses
run_pipeline(['HPV-16- und/oder -18-Positivität', 'BRCA1/2-Mutation', 'Zweitlinien- oder Drittliniensystem- bzw. -chemotherapie'])

HPV-16- und/oder -18-Positivität --> HPV-16-Positivität und/oder HPV-18-Positivität
BRCA1/2-Mutation --> BRCA1-Mutation/BRCA2-Mutation
Zweitlinien- oder Drittliniensystem- bzw. -chemotherapie --> Zweitliniensystemtherapie oder Drittliniensystemtherapie bzw. Drittlinienchemotherapie
